# Задание 4

## 1. Декоратор @cached (0.3 балла)

#### Реализуйте класс для хранения результатов выполнения функции

* max_count - максимальное число хранимых результатов. Если число результатов превышает max_count, требуется выбросить первый результат, т. е. в кеше должно храниться не более max_count последних результатов.
* продумайте архитектуру кеша так, чтобы для функций:

<code>
@cached
def f1():
    pass

@cached
def f2():
    pass
</code>    
должны иметь по max_count хранимых последних результатов, и т. д.

<b>P. S.</b>

* Считайте, что функция не имеет состояния (зависит только от передаваемых в нее аргументов).
* Храните данные так, чтобы из функции нельзя напрямую было получить закешированные результаты (только через \_\_closer\_\_).

<b>Рекомендации:</b>

* Для хранения данных используйте OrderedDict.
* Декорируйте wrapper с @functools.wraps(func)

In [1]:
from collections import OrderedDict, Counter
import functools

In [2]:
c = OrderedDict()

In [3]:
class LruCache(object):
    def __init__(self, max_count):
        self.max_count = max_count
        self.items = OrderedDict()

    def __getitem__(self, key):
        if key in self.items.keys():
            return self.items[key]
        else:
            return False

    def __setitem__(self, key, value):
        if (len(self.items) > self.max_count):
            self.items.popitem()
        self.items[key] = value
        
    def __call__(self, *args, **kwargs):
        pass

    pass

#### Реализуйте декоратор

In [4]:
def cached(max_count):
    def decorator(func):
        cache = LruCache(max_count)
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            nonlocal cache
            try:
                value = cache[(args, kwargs)]
            except:
                cache[(args, kwargs)] = func(*args, **kwargs)
            return cache[(args, kwargs)]
        return wrapper
    return decorator

#### Проверьте использование декоратора

In [5]:
@cached(20)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

@cached(20)
def fact1(n):
    if n < 2:
        return 1
    return fact1(n-1) * n

In [6]:
print(fact(5))

TypeError: unhashable type: 'dict'

#### Сравните свою реализацию с lru_cache из functools

In [ ]:
from functools import lru_cache
<your code here>

### Дополнительное задание (0.2 балла)

Дополните декоратор @cached так, чтобы не пересчитывать функцию при изменения ее состояния (например, она использовала глобальную переменную)

In [ ]:
<your code here>

## 2. Декоратор @checked (0.3 балла)

Напишите декоратор, который будет вызывать исключение (raise TypeError), если в него переданы аргументы не тех типов.

<b>P. S.</b> Разберитесь с модулем typing.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Чтобы кинуть иключение используйте конструкцию типа:
<code>
if < some_condtion >:
    raise TypeError
</code>

In [7]:
def checked(*types):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            arg_types = Counter(types)
            for arg in args:
                if (arg_types[type(arg)] <= 0):
                    raise TypeError
                arg_types[type(arg)] -= 1
            for _, value in kwargs.items():
                if (arg_types[type(value)] <= 0):
                    raise TypeError
                arg_types[type(value)] -= 1
            return func(*args, **kwargs)
        return wrapper
    return decorator

#### Проверьте использование декоратора

In [8]:
from typing import List

# Пример
@checked(str, int, list)
def strange_func(a: str, b: int, c: List):
    pass

In [9]:
strange_func('hello', 'l', ['4'])

TypeError: 

In [10]:
strange_func('hello', 7, ['4'])

## 3. Декоратор @Logger (0.4 балла)

Напишите полноценный logger для вызовов вашей функции. Декоратор должен иметь следующие опции:

* Выбор файла в который будет производиться запись: sys.stdout, sys.stderr, локальный файл (передается путь к файлу, если файла нет, то создать, иначе дописывать в конец).
* Формат записи в логера: "<i>index data time functio_name \*args \**kwargs result</i>"
* Логер должен быть один для всех функций.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Создайте отдельный класс Logger для работы с выводом данных вызовов функций в файл.

In [60]:
import sys
import time

In [81]:
class Logger(object):
    dest = None
    counter = 0
    def __init__(self, file):     
        dest = file
        
    def write(self, func_name, func_args, func_kwargs, result):
        Logger.counter += 1
        cur_date = time.strftime("%d/%m/%y")
        cur_time = time.strftime("%H:%M:%S")
        
        print(Logger.counter, cur_date, cur_time, func_name, func_args, func_kwargs, result, 
              sep='  ', file=Logger.dest)

    def __str__(self):
        pass
    
    def __repr__(self):
        pass

In [82]:
def logger(dest=sys.stdout):
    log = None
    def decorator(func):
        nonlocal log
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            nonlocal log
            if (log is None):
                log = Logger(dest)
            
            value = func(*args, **kwargs)
            log.write(func.__name__, args, kwargs, value)
            return value
           
        return wrapper
    return decorator

In [83]:
def hello(name):
    @logger(sys.stdout)
    def greet(greeting):
        return "{}, {}!".format(greeting, name)
    return greet

In [84]:
greet = hello("Natasha")
greet("Hello")
greet("Hola")
greet("Salute")

1  15/10/17  22:53:09  greet  ('Hello',)  {}  Hello, Natasha!
2  15/10/17  22:53:09  greet  ('Hola',)  {}  Hola, Natasha!
3  15/10/17  22:53:09  greet  ('Salute',)  {}  Salute, Natasha!


'Salute, Natasha!'